In [1]:
import pandas as pd
import gradio as gr
from transformers import pipeline
import numpy as np
import openai
import warnings
warnings.filterwarnings('ignore')
import os

from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")



C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
# Preference
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]

    # Write the transcription to a file
    with open("transcription_movie_selection.txt", "w") as file:
        file.write(transcription)
    
    return transcription


# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"], label="Tell me how your day was and what things you enjoy in movies!"),
    "text",
)

demo.launch(share=False)
#change to True for outside URL


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [5]:
#pure movie selection analysis with OpenAI
#preprocess txt file

with open('transcription_movie_selection.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [6]:
# Function time

def get_preference(question):
    messages = [
        {"role": "system", "content": "Determine the single emotion of the text as anger, disgust, fear, joy, neutral, sadness, or surprise and based on the provided text select three of the following genres: Action, Adventure, Animation, Comedy, Crime, Documentary, Drama, Family, Fantasy, History, Horror, Music, Mystery, Romance, Science Fiction, TV Movie, Thriller, War, and Western. Output all results in lower case font."},
        {"role": "user", "content": question}
    ]
    
    # Use the chat completions method
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    # Extract the answer from the response
    answer = response.choices[0].message.content
    return answer

def analyze_document(filename):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    print(f"{preference}")

def analyze_document(filename, movie_output):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    
    # Write the output to a text file
    with open(movie_output, 'w') as outfile:
        outfile.write(f"{preference}\n")

analyze_document('transcription_movie_selection.txt', 'movie_output.txt')

with open('movie_output.txt') as file:
    lines = file.readlines()

# Display the lines
print(text)
for line in lines:
    print(line.strip())


 I've had a really great day, but I'd really like to watch a movie that has Dogs Aliens Marshmallows and Cars
joy
action, adventure, comedy


In [7]:
#Analysis tool

# Read the recommendation_ready.csv file into the dataframe for use
movie_predictions_df = pd.read_csv("recommendation_ready.csv")

In [ ]:
#movie_predictions_df.head()

In [12]:
movie_predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6123 entries, 0 to 6122
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            6123 non-null   int64  
 1   genre_ids             6123 non-null   object 
 2   movie_id              6123 non-null   int64  
 3   popularity            6123 non-null   float64
 4   release_date          6123 non-null   object 
 5   title                 6123 non-null   object 
 6   clean_reviews         6122 non-null   object 
 7   type_Action           6123 non-null   int64  
 8   type_Adventure        6123 non-null   int64  
 9   type_Animation        6123 non-null   int64  
 10  type_Comedy           6123 non-null   int64  
 11  type_Crime            6123 non-null   int64  
 12  type_Documentary      6123 non-null   int64  
 13  type_Drama            6123 non-null   int64  
 14  type_Family           6123 non-null   int64  
 15  type_Fantasy         

In [8]:
def recommend_movies(file_path):
    # Read the file and parse the emotion and genres
    with open(file_path, 'r') as f:
        lines = f.readlines()

    emotion = lines[0]#.split(': ')[1].strip().lower()
    genres = lines[1].split(', ')#[genre.strip().lower() for genre in lines[1].split(': ')[1].split(', ')]

    # Ensure emotion column exists
   # if 'label' not in movie_predictions_df.columns:
   #     return [{"message": "Emotion label column not found in DataFrame."}]

    # Filter movies based on the predicted emotion
    emotion_movies = movie_predictions_df[movie_predictions_df['label'].str.lower() == emotion]

   # if emotion_movies.empty:
   #     return [{"message": "No movies match your emotion preference."}]

    # Initialize a DataFrame to store genre matches
    emotion_movies['genre_match_count'] = 0

    # Check and count genre matches
    for genre in genres:
        genre_column = f"type_{genre.capitalize().replace(' ', '_')}"
        if genre_column in movie_predictions_df.columns:
            emotion_movies['genre_match_count'] += emotion_movies[genre_column]
        else:
            return [{"message": f"Genre column {genre_column} not found in DataFrame."}]

    # Filter out movies that do not match any genre, if needed
    matched_movies = emotion_movies[emotion_movies['genre_match_count'] > 0]

    if matched_movies.empty:
        return [{"message": "No movies match your emotion preference."}]

    # Sort the DataFrame by genre match count (and other criteria if necessary)
    top_movies = matched_movies.sort_values(by='genre_match_count', ascending=False).head(3)

    # Prepare the final recommendations
    recommendations = [{
        "id": row['movie_id'],
        "title": row['title'],
        "poster_url": f"https://image.tmdb.org/t/p/w300{row['poster_path']}",
        "overview": row['clean_overviews'],
        "emotion": row['label'],
        "genres": ", ".join(genres),
        "matches": row['genre_match_count']
    } for index, row in top_movies.iterrows()]

    return recommendations

# Example usage
file_path = 'movie_output.txt'
recommendations = recommend_movies(file_path)
print(recommendations)


[{'message': 'Genre column type_Comedy\n not found in DataFrame.'}]
